In [1]:
import h5py
import os
import numpy as np
from tqdm import tqdm
import pickle

DATA_DIR = "../data/"
RAW_DIR = os.path.join(DATA_DIR, "raw")
PREP_DIR = os.path.join(DATA_DIR, "prep")

## Objective

    우리의 카카오 데이터를 한 데 모읍시다. 80기가가 되는데, 어떻게 줄여야 할지 고민해보아요

In [2]:
f = h5py.File(os.path.join(RAW_DIR,"train.chunk.01"),'r')

# attribute 별 memory사이즈를 계산
attr_size_dict = {}
for key in f['train'].keys():
    value = f['train'][key]
    mb_size = (value.size * value.dtype.itemsize) // (1024**2)
    attr_size_dict[key] = mb_size

tot_size = sum(attr_size_dict.values())

for key, value in attr_size_dict.items():
    print("{:10}의 메모리 size : {:4}MB({:2.2f}%)".format(
        key,value,value/tot_size*100))

bcateid   의 메모리 size :    3MB(0.03%)
brand     의 메모리 size :  124MB(1.38%)
dcateid   의 메모리 size :    3MB(0.03%)
img_feat  의 메모리 size : 7812MB(87.04%)
maker     의 메모리 size :  274MB(3.05%)
mcateid   의 메모리 size :    3MB(0.03%)
model     의 메모리 size :  142MB(1.58%)
pid       의 메모리 size :   11MB(0.12%)
price     의 메모리 size :    3MB(0.03%)
product   의 메모리 size :  583MB(6.50%)
scateid   의 메모리 size :    3MB(0.03%)
updttm    의 메모리 size :   14MB(0.16%)


Image Feature가 차지하는 메모리 분량이 87%나 된다.
즉 89기가 중 대략 77기가가 이미지 정보이고, 12기가 정도가 그 외 정보가 된다. 

이미지 Feature를 제외하고 다른 Feature들은 하나의 h5 파일로 모아보자

In [3]:
merge_dir = RAW_DIR
save_path = os.path.join(PREP_DIR,"textOnly.h5") # 여기에 저장
excludes = ['img_feat'] # img_feat 제거 

# merge 하고자 하는 데이터 셋의 파일 path
file_paths = [os.path.join(merge_dir, file_name)
              for file_name in os.listdir(merge_dir)
              if "chunk" in file_name]

# Merge하고자 하는 attribute
with h5py.File(file_paths[0], 'r') as f:
    group_name = list(f.keys())[0]
    attrs = list(f[group_name].keys())

# 저장할 attribute 리스트
attrs = list(set(attrs) - set(excludes))

with h5py.File(save_path, 'w') as write_file:
    for attr in tqdm(attrs):
        values = {}
        for file_path in file_paths:
            # 데이터를 읽어들임
            with h5py.File(file_path, 'r') as read_file:
                group_name = list(read_file.keys())[0]
                value = read_file[group_name][attr][:]

            # subset 그룹별로 저장
            if group_name not in values:
                values[group_name] = [value]
            else:
                values[group_name].append(value)

        # 나뉘어진 데이터들을 merge함
        for subset_name in values.keys():
            merge_value = np.concatenate(values[subset_name])

            if subset_name not in write_file:
                subset = write_file.create_group(subset_name)
            else:
                subset = write_file[subset_name]

            subset.create_dataset(attr, data=merge_value)

file_size = os.stat(save_path).st_size // (1024 ** 2)
print("{}에 저장되었습니다. (size : {}mb)".format(save_path, file_size))

100%|██████████| 11/11 [00:42<00:00,  3.87s/it]

../data/prep/textOnly.h5에 저장되었습니다. (size : 11899mb)


단일 파일로 이제 11기가 정도의 파일로 줄어들었다. 로컬 컴퓨터에서도 이제 데이터 탐색이 가능한 사이즈로 줄어들었다. 

직접 위의 코드를 돌리고 싶으면
bash 창에서, 

```shell
python valhalla/data.py merge ./data/raw/ ./data/prep/textOnly.h5
```

을 하면, 내부에 계층적으로 ['train','dev','test'] 식으로 group화되어 저장된다

다음에서는 이 코드를 쉽고 효율적으로 처리하는 DataLoader를 구성할 것이다